146393
228618
146128
146110
146019
146011
146003
145935
145927
145899
118377
118376
118352
118321
118220
118312
118289
118277
118264
118246

In [ ]:
#do not run if you workspace is located somewhere else
name = input("Nic or Ryan?: ")
from google.colab import drive
drive.mount('/content/drive')
import os
if name.lower() == "nic":
  os.chdir("drive/MyDrive/Colab Notebooks") # nic's
if name.lower() == "ryan":
  os.chdir("drive/MyDrive/470project3") # ryan's
os.listdir()
!ls

Nic or Ryan?: nic
Mounted at /content/drive
 Assignment5.ipynb				  Project3.ipynb
 BM25_qrel_5.txt				  project_part1.ipynb
 BM25_result_5.txt				  __pycache__
 BM25_result.txt				  qrel.txt
 combMNZ.tsv					 'Question 4.ipynb'
 embeddings.pkl					  RM3_result.txt
 Entities					 'run to qrel converter.ipynb'
 epochs.tsv					  sbert_finetuned_result_5.tsv
'Grade Calculator.ipynb'			  sbert_finetuned_result.tsv
'IR_Project2_Nicholas Drummey, Ryan Reed.ipynb'   sbert_result_5.tsv
 LGD_qrel_5.txt					  sbert_result.tsv
 LGD_result_5.txt				  SentenceBERTFineTuning.py
 LGD_result.txt					  SentenceBERTRetrieval.py
 paraphrase-MiniLM-L3-v2_finetuned_10		  TF_IDF_qrel_5.txt
'paraphrase-MiniLM-L3-v2_finetuned_10 (1)'	  TF_IDF_result_5.txt
 pd_index					  TF_IDF_result.txt
 post_parser_record.py				  Untitled0.ipynb
 Posts_Coffee.xml				  Untitled1.ipynb
 Posts.xml					  word2vec.model


In [ ]:
!pip install -U sentence-transformers
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts.xml")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.3 MB/s 
     |████████████████████████████████| 5.8 MB 33.5 MB/s 
     |████████████████████████████████| 1.3 MB 61.6 MB/s 
     |████████████████████████████████| 182 kB 79.0 MB/s 
     |████████████████████████████████| 7.6 MB 59.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=54b607288405c4473b88b5115b5b9cd98fc3b34e901b2bcd583cd52e852906d7
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


Set Querys

Generate Querys

In [ ]:
query_dic = {}
query_id_list = [146393,228618,146128,146110,146019,146011,146003,145935,145927,145899,
118377,118376,118352,118321,118220,118312,118289,118277,118264,118246]
print("the querys chosen are:")
x = 0
for query in query_id_list:
  x+=1
  question = post_reader.map_questions[query]
  print(str(x)+"- "+str(question.title))
  query_dic[query_id_list[x-1]] = str(question.title)

the querys chosen are:
1- Blender 2.8 - weathering effects
2- How to fix shading problems with this object?
3- Nurbs path doesn't go fully closed
4- Problem with bone
5- Scaling faces with drivers
6- How do I render Glowing text effect
7- Creating a hole out of an custom shape
8- Brick texture with multiple colors
9- How to Dribble soccer ball?
10- Masking selected vertices for sculpting
11- How to use Modular Trees
12- Netrender addon missing
13- drawing a belt around a gear
14- Constrain vertex positions to moving objects?
15- Deformation of particle objects
16- Cannot change smoke color?
17- Mix shader is not working
18- Scale deformation when rotating imported FBX
19- cubic trefoil knot
20- How to get the absolute path?


Step 1: Create Training Data

In [ ]:
import random
import re

training_triples = {"query":[],"text":[], "label":[]}
title_list = {}
answers = []
answer_ids = []
#finds answer ids and saves sentences and ids to seperate lists
for answer_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[answer_id]
  answer_ids.append(answer_id)
  answers.append(re.sub(r'[^a-z]', ' ', answer.body.lower().strip()))

#creates list of question titles
print("Creating Training Triples...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  if question.accepted_answer_id != None:
    text = question.body.lower().strip()
    title_list[question_id] = text.replace('<p>','').replace('</p>','')

#removes query titles from list
for key in query_id_list:
  if key in title_list:
    title_list.pop(key)

#creates training dictionary
even_check = 0
for entry in title_list:
  if (even_check % 2) == 0:
    question = post_reader.map_questions[entry]
    answer = post_reader.map_just_answers[question.accepted_answer_id]
    text = answer.body.lower()
    training_triples.get("query").append(str(title_list[entry]))
    training_triples.get("text").append(str(text))
    training_triples.get("label").append(str(1))
  else: 
    random_id = random.choice(answer_ids)
    question = post_reader.map_questions[entry]
    answer = post_reader.map_just_answers[random_id]
    text = answer.body.lower()
    training_triples.get("query").append(str(title_list[entry]))
    training_triples.get("text").append(str(text))
    training_triples.get("label").append(str(0))
  even_check += 1
print("Complete")

Creating Training Triples...
Complete


Step 2: Pretrained

In [ ]:
import os
import csv

from sentence_transformers import SentenceTransformer
from sentence_transformers import util


# Setting GPU card
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def read_corpus(post_file_path):
    # Takes in Posts.xml file and returns dictionary of {answer id: answer body}
    # This is the collection you will search over
    post_parser = PostParserRecord(post_file_path)
    dict_answer_id_answer = {}
    for answer_id in post_parser.map_just_answers:
        dict_answer_id_answer[answer_id] = post_parser.map_just_answers[answer_id].body
    return dict_answer_id_answer


def retrieval(model_name, post_file_path):
    final_result = {}
    print("model loaded")
    model = SentenceTransformer(model_name)
    "This is an important part"
    candidates = read_corpus(post_file_path)
    print("corpus read")
    corpus_embeddings = model.encode(list(candidates.values()), convert_to_tensor=True)
    print("corpus encoded")

    queries = query_dic
    # You will write your own method to fill this dictionary as {query_id: query} where query is the query text

    for topic_id in queries:
        temp_dic = {}
        query = queries[topic_id]
        query_embedding = model.encode(query, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=1000)
        hits = hits[0]  # Get the hits for the first query
        for hit in hits:
            index = hit['corpus_id']
            answer_id = list(candidates.keys())[index]
            score = hit['score']
            temp_dic[answer_id] = score
        final_result[topic_id] = temp_dic
    return final_result


model_name = "paraphrase-MiniLM-L3-v2"
retrieval_results = retrieval(model_name, "Posts.xml")

result_file_path = "sbert_result.tsv"
run_name = "sbrt"
# Saving the top-1000 results
with open(result_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for topic_id in retrieval_results:
        result_map = retrieval_results[topic_id]
        result_map = dict(sorted(result_map.items(), key=lambda item: item[1], reverse=True))
        rank = 1
        for post_id in result_map:
            score = result_map[post_id]
            csv_writer.writerow([topic_id, "Q0",  post_id, str(rank), str(score), run_name])
            rank += 1
            if rank > 1000:
                break

model loaded


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

corpus read
corpus encoded


Step 3: Trained

In [ ]:
import csv
import os
import random
import re


#from Entity_Parser_Record.post_parser_record import PostParserRecord
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util, models, evaluation
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def get_list_of_questions_with_accepted_answer(post_parser):
    # This method takes in post_parser and returns list of question ids that has an accepted answer
    lst_question_ids = []
    for question_id in post_parser.map_questions:
        question = post_parser.map_questions[question_id]
        if question.accepted_answer_id is not None:
            lst_question_ids.append(question_id)
    return lst_question_ids


def read_corpus(post_file_path):
    # This methods takes in the file path of Posts.xml file and returns a list of triplets
    # These triplets are in form of (query, text, label) where we have two labels of 0 and 1 for negative and
    # negative candidates
    training_triplets = {}
    post_parser = PostParserRecord(post_file_path)
    list_questions_with_accepted_answer = get_list_of_questions_with_accepted_answer(post_parser)

    # This list should be filled with the question ids that you have used for evaluating your systems
    # Write your own method to fill this list
    except_these_question_ids = query_id_list
    answer_ids = list(post_parser.map_just_answers.keys())
    for exclude_id in except_these_question_ids:
      if exclude_id in answer_ids:
        answer_ids.remove(exclude_id)
        
    random.shuffle(answer_ids)
    random.seed = 4
    for question_id in list_questions_with_accepted_answer:
        question = post_parser.map_questions[question_id]
        query = question.title
        positive_text = post_parser.map_just_answers[question.accepted_answer_id].body
        # You need to include your pre-processing of text code here: removing html tags
        positive_text = re.sub('<.*?>', '', positive_text)

        lst_of_answer_ids_to_question = []
        for answer in question.answers:
            lst_of_answer_ids_to_question.append(answer.post_id)

        #random.shuffle(answer_ids)
        for answer_id in answer_ids:
            if answer_id in lst_of_answer_ids_to_question:
                continue
            else:
                negative_text = post_parser.map_just_answers[answer_id].body
                break
        if 0 in training_triplets:
            training_triplets[0].append([query, negative_text, 0])
        else:
            training_triplets[0] = [[query, negative_text, 0]]
        if 1 in training_triplets:
            training_triplets[1].append([query, positive_text, 1])
        else:
            training_triplets[1] = [[query, positive_text, 1]]
    return training_triplets


def split_data(data):
    # takes in list as the input data and return split of data (splitting into 10 pieces)
    length = int(len(data) / 10)  # length of each fold
    pieces = []
    for i in range(9):
        pieces.append(data[i * length: (i + 1) * length])
    pieces.append(data[9 * length:len(data)])
    return pieces


def fine_tuning(post_xml_file_path):
    # You will define the pre-trained model that you aim to use
    model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

    # The name of model to save after fine-tuning
    MODEL = "paraphrase-MiniLM-L3-v2_finetuned_100"

    # This fine-tuning uses two loss functions and for each we consider one list to save the triplets
    train_samples_MultipleNegativesRankingLoss = []
    train_samples_ConstrativeLoss = []

    # parallel lists used for defining the validation set
    evaluator_samples_1 =[]
    evaluator_samples_2 = []
    evaluator_samples_score = []

    # Parameters for fine-tuning
    num_epochs = 20
    margin = 0.5
    train_batch_size = 64
    model.max_seq_length = 128
    distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

    # Reading the post file and extracting the samples for fine-tuning
    dic_instances = read_corpus(post_xml_file_path)

    #################################
    # Preparing the data for Sentence-BERT in the required format
    for label in dic_instances:
        lst_instances = dic_instances[label]
        # Randomly choosing the validation split
        random.shuffle(lst_instances)
        splits = split_data(lst_instances)
        # The last split is used as validation set
        validation = splits[-1]
        training = splits[:-1]

        # Preparing the training set
        for split in training:
            for triples in split:
                query = triples[0]
                candidate = triples[1]
                label = triples[2]
                train_samples_ConstrativeLoss.append(InputExample(texts=[query, candidate], label=label))
                train_samples_MultipleNegativesRankingLoss.append(
                        InputExample(texts=[query, candidate], label=label))
        # Preparing the validation set
        for triples in validation:
            evaluator_samples_1.append(triples[0])
            evaluator_samples_2.append(triples[1])
            evaluator_samples_score.append(triples[2])

    # Create data loader and loss for MultipleNegativesRankingLoss
    train_dataset_MultipleNegativesRankingLoss = SentencesDataset(train_samples_MultipleNegativesRankingLoss,
                                                                  model=model)
    train_dataloader_MultipleNegativesRankingLoss = DataLoader(train_dataset_MultipleNegativesRankingLoss, shuffle=True,
                                                               batch_size=train_batch_size)
    train_loss_MultipleNegativesRankingLoss = losses.MultipleNegativesRankingLoss(model)

    # Create data loader and loss for OnlineContrastiveLoss
    train_dataset_ConstrativeLoss = SentencesDataset(train_samples_ConstrativeLoss, model=model)
    train_dataloader_ConstrativeLoss = DataLoader(train_dataset_ConstrativeLoss, shuffle=True,
                                                  batch_size=train_batch_size)
    train_loss_ConstrativeLoss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric,
                                                              margin=margin)

    # Setting up the validation set
    evaluator = evaluation.EmbeddingSimilarityEvaluator(evaluator_samples_1, evaluator_samples_2, evaluator_samples_score, write_csv="evaluation_epoch.csv")

    # Fine-tune the model
    model.fit(
        train_objectives=[(train_dataloader_MultipleNegativesRankingLoss, train_loss_MultipleNegativesRankingLoss),
                          (train_dataloader_ConstrativeLoss, train_loss_ConstrativeLoss)],
        evaluator=evaluator,
        epochs=num_epochs,
        warmup_steps=1000,
        output_path=MODEL,
        show_progress_bar=True,
        callback=callback
        )


def callback(score, epoch, steps):
    # This method is used to write the loss for each epoch on the validation set
    csv_writer_Epochs.writerow([score, epoch, steps])


epoch_csv_file = "epochs.tsv"
post_xml_file = "Posts.xml"
with open(epoch_csv_file, mode='w', newline='') as csv_file:
    csv_writer_Epochs = csv.writer(csv_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    fine_tuning(post_xml_file)



ModuleNotFoundError: ignored

Search for trained model

In [ ]:
import os
import csv

from sentence_transformers import SentenceTransformer
from sentence_transformers import util


# Setting GPU card
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def read_corpus(post_file_path):
    # Takes in Posts.xml file and returns dictionary of {answer id: answer body}
    # This is the collection you will search over
    post_parser = PostParserRecord(post_file_path)
    dict_answer_id_answer = {}
    for answer_id in post_parser.map_just_answers:
        dict_answer_id_answer[answer_id] = post_parser.map_just_answers[answer_id].body
    return dict_answer_id_answer


def retrieval(model_name, post_file_path):
    final_result = {}
    print("model loaded")
    model = SentenceTransformer(model_name)
    "This is an important part"
    candidates = read_corpus(post_file_path)
    print("corpus read")
    corpus_embeddings = model.encode(list(candidates.values()), convert_to_tensor=True)
    print("corpus encoded")

    queries = query_dic
    # You will write your own method to fill this dictionary as {query_id: query} where query is the query text

    for topic_id in queries:
        temp_dic = {}
        query = queries[topic_id]
        query_embedding = model.encode(query, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=1000)
        hits = hits[0]  # Get the hits for the first query
        for hit in hits:
            index = hit['corpus_id']
            answer_id = list(candidates.keys())[index]
            score = hit['score']
            temp_dic[answer_id] = score
        final_result[topic_id] = temp_dic
    return final_result


model_name = "paraphrase-MiniLM-L3-v2_finetuned_10"
retrieval_results = retrieval(model_name, "Posts.xml")

result_file_path = "sbert_finetuned_result.tsv"
run_name = "sbrt"
# Saving the top-1000 results
with open(result_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for topic_id in retrieval_results:
        result_map = retrieval_results[topic_id]
        result_map = dict(sorted(result_map.items(), key=lambda item: item[1], reverse=True))
        rank = 1
        for post_id in result_map:
            score = result_map[post_id]
            csv_writer.writerow([topic_id, "Q0",  post_id, str(rank), str(score), run_name])
            rank += 1
            if rank > 1000:
                break

model loaded
corpus read
corpus encoded
